In [10]:
import pandas as pd
from geopy.distance import geodesic

def addClosestStation(input_file, output_file, sep, lat_col, lon_col):
    
    stations = pd.read_csv('data/air/stations.csv', sep=',', encoding='latin-1')
    locations = pd.read_csv(input_file, sep=sep, encoding='latin-1')
    
    locations["closestStation"] = None

    for i in range(len(locations)):
        min_dist = 100000
        closest = ""
        location = locations.loc[i]
        coords_location = (float(location[lat_col]), float(location[lon_col]))
        for key, station in stations.iterrows():
            coords_station = (float(station["lat"]), float(station["lon"]))

            dist = geodesic(coords_location, coords_station).miles

            if dist < min_dist:
                min_dist, closest = dist, station["id"]

        locations.set_value(i, 'closestStation', closest)

    locations.to_csv(output_file, sep=';', encoding='utf-8')

#addClosestStation("ProcessedData/schools.csv", "ProcessedData/schools_cs.csv", ";", "LATITUD", "LONGITUD")  
#addClosestStation("ProcessedData/elderlyareas.csv", "ProcessedData/elderlyareas_cs.csv", ",", "LATITUD", "LONGITUD")
#addClosestStation("ProcessedData/sportcenters.csv", "ProcessedData/sportcenters_cs.csv", ",", "LATITUD", "LONGITUD")

In [38]:
import rdflib
import rdfextras

lv2 = rdflib.Namespace('http://safeair.es/project/ontology/SafeAir#')
path = "RDFs/stations.ttl" 
plugin = rdflib.URIRef("http://safeair.es/project/resources/")

model = rdflib.ConjunctiveGraph()

path = "RDFs/stations.ttl" 
model.parse(path, format='ttl')

path = "RDFs/schools.ttl" 
model.parse(path, format='ttl')

qres = model.query("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix safeair: <http://safeare.es/project/ontology/SafeAir#>

SELECT ?p ?s
WHERE {
 ?p a safeair:School;
  safeair:isClosestTo "28079004"^^xsd:int .
}
""")

for row in qres:
    print(row)

(rdflib.term.URIRef('http://safeair.es/project/resources/School/5633199'), rdflib.term.URIRef('http://safeair.es/project/resources/Station/28079004'))
(rdflib.term.URIRef('http://safeair.es/project/resources/School/100359'), rdflib.term.URIRef('http://safeair.es/project/resources/Station/28079004'))
